In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv('data.csv')

In [3]:
data

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11909,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,46120
11910,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,56670
11911,Acura,ZDX,2012,premium unleaded (required),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50620
11912,Acura,ZDX,2013,premium unleaded (recommended),300.0,6.0,AUTOMATIC,all wheel drive,4.0,"Crossover,Hatchback,Luxury",Midsize,4dr Hatchback,23,16,204,50920


In [4]:
cols = ['Make',
    'Model',
    'Year',
    'Engine HP',
    'Engine Cylinders',
    'Transmission Type',
    'Vehicle Style',
    'highway MPG',
    'city mpg'
]

In [5]:
data.columns = data.columns.str.replace(' ', '_').str.lower()

In [6]:
data.isnull().sum()

make                    0
model                   0
year                    0
engine_fuel_type        3
engine_hp              69
engine_cylinders       30
transmission_type       0
driven_wheels           0
number_of_doors         6
market_category      3742
vehicle_size            0
vehicle_style           0
highway_mpg             0
city_mpg                0
popularity              0
msrp                    0
dtype: int64

In [7]:
data = data.fillna(0)

In [66]:
data['price'] = data['msrp'] 
del data['msrp']

In [9]:
data['transmission_type'].value_counts()

AUTOMATIC           8266
MANUAL              2935
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
Name: transmission_type, dtype: int64

- question 1: Automatic is the most frequent transmission_type

In [10]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

C:\Users\refael.y\AppData\Local\Temp\ipykernel_19496\1398619703.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = data.corr()


,year,engine_hp,engine_cylinders,number_of_doors,highway_mpg,city_mpg,popularity,msrp,price
year,1.000000,0.338714,-0.040708,0.261431,0.258240,0.198171,0.073049,0.227590,0.227590
engine_hp,0.338714,1.000000,0.774851,-0.104588,-0.415707,-0.424918,0.031409,0.650095,0.650095
engine_cylinders,-0.040708,0.774851,1.000000,-0.136669,-0.614541,-0.587306,0.045226,0.526274,0.526274
number_of_doors,0.261431,-0.104588,-0.136669,1.000000,0.102457,0.104597,-0.048177,-0.128627,-0.128627
highway_mpg,0.258240,-0.415707,-0.614541,0.102457,1.000000,0.886829,-0.020991,-0.160043,-0.160043
city_mpg,0.198171,-0.424918,-0.587306,0.104597,0.886829,1.000000,-0.003217,-0.157676,-0.157676
popularity,0.073049,0.031409,0.045226,-0.048177,-0.020991,-0.003217,1.000000,-0.048476,-0.048476
msrp,0.227590,0.650095,0.526274,-0.128627,-0.160043,-0.157676,-0.048476,1.000000,1.000000
price,0.227590,0.650095,0.526274,-0.128627,-0.160043,-0.157676,-0.048476,1.000000,1.000000


- question 2: highwaympg and city_mpg have the biggest correlation

In [11]:
data['above_price'] = data['price'] > data['price'].mean()

In [12]:
data['above_price'] = data['above_price'].astype(int)

In [13]:
from sklearn.model_selection import train_test_split 

In [67]:
df_full_train, df_test = train_test_split(data, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

y_train = df_train.above_price.values
y_val = df_val.above_price.values
y_test = df_test.above_price.values


In [36]:
df_train.dtypes

make                  object
model                 object
year                   int64
engine_fuel_type      object
engine_hp            float64
engine_cylinders     float64
transmission_type     object
driven_wheels         object
number_of_doors      float64
market_category       object
vehicle_size          object
vehicle_style         object
highway_mpg            int64
city_mpg               int64
popularity             int64
msrp                   int64
price                  int64
above_price            int32
dtype: object

In [37]:
df_train.dtypes

cat_cols = ['make', 'model', 'transmission_type', 'driven_wheels', 'vehicle_size', 'vehicle_style']
cat_df_train = df_train[cat_cols]

In [58]:
from sklearn.metrics import mutual_info_score
for cat_col in cat_cols:
    score = mutual_info_score(df_train.above_price, df_train[cat_col])
    print(cat_col, round(score,2))

make 0.24
model 0.46
transmission_type 0.02
driven_wheels 0.1
vehicle_size 0.04
vehicle_style 0.08


- question 3: transmission_type has the lowest MI score with above_price column

In [68]:
from sklearn.feature_extraction import DictVectorizer

df_train = df_train.drop(['above_price', 'price'], axis=1)
df_val = df_val.drop(['above_price', 'price'], axis=1)
df_test = df_test.drop(['above_price', 'price'], axis=1)

print(df_train.head())

dict_train = df_train.to_dict(orient='records')
dict_val = df_val.to_dict(orient='records')
v = DictVectorizer()
X = v.fit_transform(dict_train) 
X_val = v.transform(dict_val)

              make     model  year                engine_fuel_type  engine_hp  \
3972    Mitsubishi  Endeavor  2011  premium unleaded (recommended)      225.0   
1997           Kia   Borrego  2009                regular unleaded      276.0   
5216   Lamborghini  Gallardo  2012     premium unleaded (required)      570.0   
2805     Chevrolet  Colorado  2016                regular unleaded      200.0   
11369      Pontiac      Vibe  2009                regular unleaded      158.0   

       engine_cylinders transmission_type     driven_wheels  number_of_doors  \
3972                6.0         AUTOMATIC   all wheel drive              4.0   
1997                6.0         AUTOMATIC  rear wheel drive              4.0   
5216               10.0            MANUAL   all wheel drive              2.0   
2805                4.0         AUTOMATIC  rear wheel drive              4.0   
11369               4.0         AUTOMATIC   all wheel drive              4.0   

                             mar

In [69]:
from sklearn.linear_model import LogisticRegression 
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X, y_train)
y_pred = model.predict(X_val)

In [70]:
from sklearn.metrics import accuracy_score

In [72]:
base_score = accuracy_score(y_val, y_pred)
base_score = round(base_score, 2)
print(f'Score is {base_score}')

Score is 0.94


- question4: score is `0.94`

In [73]:
from sklearn.linear_model import LogisticRegression

for col in df_train:
    model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
    df_train_temp = df_train.copy()
    df_val_temp = df_val.copy()
    
    del df_train_temp[col]
    del df_val_temp[col]
    
    dict_train = df_train_temp.to_dict(orient='records')
    dict_val = df_val_temp.to_dict(orient='records')
    v = DictVectorizer()
    X = v.fit_transform(dict_train) 
    X_val = v.transform(dict_val)
    
    model.fit(X, y_train)
    y_pred = model.predict(X_val)
    score = accuracy_score(y_val, y_pred)
    print(col , score, base_score - score)

make 0.9353755770037767 0.004624422996223232
model 0.9307595467897608 0.009240453210239141
year 0.9467058329836341 -0.006705832983634141
engine_fuel_type 0.9391523289970626 0.0008476710029373669
engine_hp 0.9441879983214435 -0.0041879983214435645
engine_cylinders 0.9391523289970626 0.0008476710029373669
transmission_type 0.946286193873269 -0.006286193873269008
driven_wheels 0.9437683592110785 -0.0037683592110785424
number_of_doors 0.9467058329836341 -0.006705832983634141
market_category 0.9349559378934117 0.005044062106588254
vehicle_size 0.9282417121275703 0.011758287872429607
vehicle_style 0.9467058329836341 -0.006705832983634141
highway_mpg 0.9450272765421738 -0.005027276542173831
city_mpg 0.950902224087285 -0.010902224087285028
popularity 0.9479647503147294 -0.00796475031472943


- question 5: the col with the smallest diff is `engine_hp` with `-0.0041879983214435645` 

In [80]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
df_full_train, df_test = train_test_split(data, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

alphas = [0, 0.01, 0.1, 1, 10]
y_train = np.log1p(df_train.price)
y_val = np.log1p(df_val.price)
y_test = np.log1p(df_test.price)

df_train = df_train.drop(['above_price', 'price'], axis=1)
df_val = df_val.drop(['above_price', 'price'], axis=1)
df_test = df_test.drop(['above_price', 'price'], axis=1)


dict_train = df_train_temp.to_dict(orient='records')
dict_val = df_val_temp.to_dict(orient='records')
v = DictVectorizer()
X = v.fit_transform(dict_train) 
X_val = v.transform(dict_val)

for alpha in alphas:
    model = Ridge(alpha=alpha, solver='sag', random_state=42)
    model.fit(X, y_train)
    y_pred = model.predict(X_val)
    rmse_score = mean_squared_error(y_val, y_pred)
    print(alpha, rmse_score)

              make     model  year                engine_fuel_type  engine_hp  \
3972    Mitsubishi  Endeavor  2011  premium unleaded (recommended)      225.0   
1997           Kia   Borrego  2009                regular unleaded      276.0   
5216   Lamborghini  Gallardo  2012     premium unleaded (required)      570.0   
2805     Chevrolet  Colorado  2016                regular unleaded      200.0   
11369      Pontiac      Vibe  2009                regular unleaded      158.0   

       engine_cylinders transmission_type     driven_wheels  number_of_doors  \
3972                6.0         AUTOMATIC   all wheel drive              4.0   
1997                6.0         AUTOMATIC  rear wheel drive              4.0   
5216               10.0            MANUAL   all wheel drive              2.0   
2805                4.0         AUTOMATIC  rear wheel drive              4.0   
11369               4.0         AUTOMATIC   all wheel drive              4.0   

                             mar

- question 6: the best alpha is: 